Decided to look at the US coastal AIS data for 01.01.23

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
#Holder store filer utenfor wsl-en
os.chdir('/mnt/c/users/alexa/Downloads')
os.getcwd()

'/mnt/c/users/alexa/Downloads'

In [ ]:
INPUT_CSV = "AIS_2023_01_01.csv"
OUT_CSV = "AIS_midlertidig.csv"
CHUNKSIZE = 100_000

# Krav for aisdb pakken
list_of_headers_ = ["MMSI","Message_ID","Repeat_indicator","Time","Millisecond","Region","Country","Base_station","Online_data","Group_code","Sequence_ID",
                    "Channel","Data_length","Vessel_Name","Call_sign","IMO","Ship_Type","Dimension_to_Bow","Dimension_to_stern","Dimension_to_port",
                    "Dimension_to_starboard","Draught","Destination","AIS_version","Navigational_status","ROT","SOG","Accuracy","Longitude","Latitude",
                    "COG","Heading","Regional","Maneuver","RAIM_flag","Communication_flag","Communication_state","UTC_year","UTC_month","UTC_day","UTC_hour",
                    "UTC_minute","UTC_second","Fixing_device","Transmission_control","ETA_month","ETA_day","ETA_hour","ETA_minute","Sequence","Destination_ID",
                    "Retransmit_flag","Country_code","Functional_ID","Data","Destination_ID_1","Sequence_1","Destination_ID_2","Sequence_2","Destination_ID_3",
                    "Sequence_3","Destination_ID_4","Sequence_4","Altitude","Altitude_sensor","Data_terminal","Mode","Safety_text","Non-standard_bits",
                    "Name_extension","Name_extension_padding","Message_ID_1_1","Offset_1_1","Message_ID_1_2","Offset_1_2","Message_ID_2_1","Offset_2_1",
                    "Destination_ID_A","Offset_A","Increment_A","Destination_ID_B","offsetB","incrementB","data_msg_type","station_ID","Z_count","num_data_words",
                    "health","unit_flag","display","DSC","band","msg22","offset1","num_slots1","timeout1","Increment_1","Offset_2","Number_slots_2","Timeout_2",
                    "Increment_2","Offset_3","Number_slots_3","Timeout_3","Increment_3","Offset_4","Number_slots_4","Timeout_4","Increment_4","ATON_type",
                    "ATON_name","off_position","ATON_status","Virtual_ATON","Channel_A","Channel_B","Tx_Rx_mode","Power","Message_indicator","Channel_A_bandwidth",
                    "Channel_B_bandwidth","Transzone_size","Longitude_1","Latitude_1","Longitude_2","Latitude_2","Station_Type","Report_Interval","Quiet_Time",
                    "Part_Number","Vendor_ID","Mother_ship_MMSI","Destination_indicator","Binary_flag","GNSS_status","spare","spare2","spare3","spare4"
                    ]


SOURCE_MAP = {
    "MMSI": "MMSI",
    "BaseDateTime": "Time",
    "LAT": "Latitude",
    "LON": "Longitude",
    "VesselName": "Vessel_Name",
    "CallSign": "Call_sign",
    "VesselType": "Ship_Type",
    "Status": "Navigational_status",
    "Draft": "Draught",
}

def make_df_new(df_src: pd.DataFrame) -> pd.DataFrame:
    df_new = pd.DataFrame(index=df_src.index, columns=list_of_headers_, dtype="object")

    for src, dst in SOURCE_MAP.items():
        if src in df_src.columns:
            df_new[dst] = df_src[src]

    df_new["MMSI"] = pd.to_numeric(df_new["MMSI"], errors="coerce").astype("Int64")
    df_new["Time"] = pd.to_datetime(df_new["Time"], errors="coerce").dt.strftime("%Y%m%d_%H%M%S")
    df_new["Latitude"] = pd.to_numeric(df_new["Latitude"], errors="coerce")
    df_new["Longitude"] = pd.to_numeric(df_new["Longitude"], errors="coerce")

    df_new["Ship_Type"] = pd.to_numeric(df_new["Ship_Type"], errors="coerce").fillna(0).astype("Int64")
    df_new["Message_ID"] = 1
    df_new["Millisecond"] = 0

    df_new = df_new.dropna(subset=["MMSI", "Time", "Latitude", "Longitude"])
    return df_new

def main():
    pd.DataFrame(columns=list_of_headers_).to_csv(OUT_CSV, index=False)

    seen_static = set()
    total_static = 0
    total_dynamic = 0

    # 1: Statisk
    for chunk in pd.read_csv(INPUT_CSV, chunksize=CHUNKSIZE, low_memory=False):
        df_new = make_df_new(chunk)

        df_static = df_new[df_new["Ship_Type"].notna() & (df_new["Ship_Type"].astype("Int64") != 0)].copy()
        if df_static.empty:
            continue

        df_static = df_static.drop_duplicates(subset="MMSI", keep="first")

        keep_rows = []
        for m in df_static["MMSI"].tolist():
            mi = int(m) if pd.notna(m) else None
            if mi is None or mi in seen_static:
                keep_rows.append(False)
            else:
                seen_static.add(mi)
                keep_rows.append(True)

        df_static = df_static.loc[keep_rows].copy()
        if df_static.empty:
            continue

        df_static["Message_ID"] = 5
        total_static += len(df_static)

        df_static.to_csv(OUT_CSV, mode="a", index=False, header=False)

    # 2: Dynamisk
    for chunk in pd.read_csv(INPUT_CSV, chunksize=CHUNKSIZE, low_memory=False):
        df_new = make_df_new(chunk)
        if df_new.empty:
            continue
        total_dynamic += len(df_new)
        df_new.to_csv(OUT_CSV, mode="a", index=False, header=False)

    print("Done:", OUT_CSV)
    print("Static rows written:", total_static)
    print("Dynamic rows written:", total_dynamic)

main()

Done: AIS_midlertidig.csv
Static rows written: 15738
Dynamic rows written: 8156509


In [26]:
import aisdb
from pathlib import Path

INPUT_CSV = "/mnt/c/Users/alexa/Downloads/AIS_midlertidig.csv"
DBPATH = str(Path("/home/linex") / "01_01_23_AIS.db")

with aisdb.SQLiteDBConn(dbpath=DBPATH) as dbconn:
    aisdb.decode_msgs(
        filepaths=[INPUT_CSV],
        dbconn=dbconn,
        source="NOAA",
        verbose=True
    )

print("DB created at:", DBPATH)

generating file checksums...
checking file dates...
creating tables and dropping table indexes...
Memory: 5.28GB remaining.  CPUs: 16.  Average file size: 3337.57MB  Spawning 4 workers
saving checksums...
processing /mnt/c/Users/alexa/Downloads/AIS_midlertidig.csv
AIS_midlertidig.csv                                              count: 8172247    elapsed:  328.76s    rate:    24858 msgs/s
cleaning temporary data...
aggregating static reports into static_202301_aggregate...
DB created at: /home/linex/01_01_23_AIS.db
